In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
%matplotlib inline

In [124]:
df = pd.read_csv('nhl_stats.csv', index_col=0)
df.head()

,team,date,points,RW,ROW,SOW,goals,goals_against,power_play,penalty_kill,net_ppp,net_pkp,shots,shots_against,FOWp
0,Philadelphia Flyers,2010/10/07@ PIT,2.0,1.0,1.0,0.0,3.0,2.0,25.0,80.0,25.0,100.0,27.0,31.0,39.3
1,Pittsburgh Penguins,2010/10/07vs PHI,0.0,0.0,0.0,0.0,2.0,3.0,20.0,75.0,0.0,75.0,31.0,27.0,60.7
2,Montréal Canadiens,2010/10/07@ TOR,0.0,0.0,0.0,0.0,2.0,3.0,0.0,100.0,0.0,100.0,28.0,24.0,53.5
3,Toronto Maple Leafs,2010/10/07vs MTL,2.0,1.0,1.0,0.0,3.0,2.0,0.0,100.0,0.0,100.0,24.0,28.0,46.5
4,Carolina Hurricanes,2010/10/07@ MIN,2.0,1.0,1.0,0.0,4.0,3.0,40.0,60.0,40.0,60.0,31.0,29.0,38.8


In [125]:
df.dtypes

team              object
date              object
points           float64
RW               float64
ROW              float64
SOW              float64
goals            float64
goals_against    float64
power_play        object
penalty_kill      object
net_ppp           object
net_pkp           object
shots            float64
shots_against    float64
FOWp             float64
dtype: object

In [126]:
df=df.dropna()

In [127]:
df.tail()

,team,date,points,RW,ROW,SOW,goals,goals_against,power_play,penalty_kill,net_ppp,net_pkp,shots,shots_against,FOWp
25139,Minnesota Wild,2021/05/13@ STL,0.0,0.0,0.0,0.0,3.0,7.0,0.0,25.0,0.0,25.0,20.0,27.0,49.0
25140,Toronto Maple Leafs,2021/05/14@ WPG,0.0,0.0,0.0,0.0,2.0,4.0,0.0,100.0,0.0,100.0,36.0,24.0,52.2
25141,Winnipeg Jets,2021/05/14vs TOR,2.0,1.0,1.0,0.0,4.0,2.0,0.0,100.0,0.0,100.0,24.0,36.0,47.8
25142,Edmonton Oilers,2021/05/15vs VAN,0.0,0.0,0.0,0.0,1.0,4.0,0.0,100.0,-33.3,100.0,32.0,41.0,55.1
25143,Vancouver Canucks,2021/05/15@ EDM,2.0,1.0,1.0,0.0,4.0,1.0,0.0,100.0,0.0,133.3,41.0,32.0,44.9


In [128]:
df['opponent'] = df.date.apply(lambda x: x.split()[1])

In [129]:
df['opponent'].unique()

array(['PIT', 'PHI', 'TOR', 'MTL', 'MIN', 'COL', 'EDM', 'CHI', 'CGY',
       'CAR', 'DAL', 'OTT', 'BUF', 'WSH', 'ATL', 'ANA', 'DET', 'NJD',
       'CBJ', 'SJS', 'STL', 'PHX', 'NYR', 'TBL', 'LAK', 'NSH', 'NYI',
       'VAN', 'BOS', 'FLA', 'WPG', 'ARI', 'VGK'], dtype=object)

In [130]:
df['team'].unique()

array(['Philadelphia Flyers', 'Pittsburgh Penguins', 'Montréal Canadiens',
       'Toronto Maple Leafs', 'Carolina Hurricanes', 'Chicago Blackhawks',
       'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers',
       'Minnesota Wild', 'New Jersey Devils', 'Buffalo Sabres',
       'Ottawa Senators', 'Atlanta Thrashers', 'Washington Capitals',
       'Detroit Red Wings', 'Anaheim Ducks', 'Dallas Stars',
       'San Jose Sharks', 'Columbus Blue Jackets', 'New York Islanders',
       'New York Rangers', 'Boston Bruins', 'Tampa Bay Lightning',
       'Nashville Predators', 'St. Louis Blues', 'Vancouver Canucks',
       'Los Angeles Kings', 'Phoenix Coyotes', 'Florida Panthers',
       'Winnipeg Jets', 'Arizona Coyotes', 'Vegas Golden Knights'],
      dtype=object)

In [131]:
abb_to_team = {'PIT':'Pittsburgh Penguins', 'PHI':'Philadelphia Flyers',
'TOR':'Toronto Maple Leafs', 'MTL':'Montréal Canadiens', 'MIN':'Minnesota Wild',
'COL':'Colorado Avalanche', 'EDM':'Edmonton Oilers', 'CHI':'Chicago Blackhawks',
'CGY':'Calgary Flames', 'CAR':'Carolina Hurricanes', 'DAL':'Dallas Stars',
'OTT':'Ottawa Senators', 'BUF':'Buffalo Sabres', 'WSH':'Washington Capitals',
'ATL':'Atlanta Thrashers', 'ANA':'Anaheim Ducks', 'DET':'Detroit Red Wings',
'NJD':'New Jersey Devils','CBJ':'Columbus Blue Jackets', 'SJS':'San Jose Sharks',
'STL':'St. Louis Blues', 'PHX':'Phoenix Coyotes', 'NYR':'New York Rangers',
'TBL':'Tampa Bay Lightning', 'LAK':'Los Angeles Kings', 'NSH':'Nashville Predators',
'NYI':'New York Islanders', 'VAN':'Vancouver Canucks', 'BOS':'Boston Bruins',
'FLA':'Florida Panthers', 'WPG':'Winnipeg Jets', 'ARI':'Arizona Coyotes',
'VGK':'Vegas Golden Knights', 'SEA':'Seattle Kraken'}

In [132]:
df['date'] = df['date'].apply(lambda x: pd.to_datetime(x[0:10], format='%Y/%m/%d'))
df['opponent'] = df['opponent'].apply(lambda x: abb_to_team[x])
df.head()

,team,date,points,RW,ROW,SOW,goals,goals_against,power_play,penalty_kill,net_ppp,net_pkp,shots,shots_against,FOWp,opponent
0,Philadelphia Flyers,2010-10-07,2.0,1.0,1.0,0.0,3.0,2.0,25.0,80.0,25.0,100.0,27.0,31.0,39.3,Pittsburgh Penguins
1,Pittsburgh Penguins,2010-10-07,0.0,0.0,0.0,0.0,2.0,3.0,20.0,75.0,0.0,75.0,31.0,27.0,60.7,Philadelphia Flyers
2,Montréal Canadiens,2010-10-07,0.0,0.0,0.0,0.0,2.0,3.0,0.0,100.0,0.0,100.0,28.0,24.0,53.5,Toronto Maple Leafs
3,Toronto Maple Leafs,2010-10-07,2.0,1.0,1.0,0.0,3.0,2.0,0.0,100.0,0.0,100.0,24.0,28.0,46.5,Montréal Canadiens
4,Carolina Hurricanes,2010-10-07,2.0,1.0,1.0,0.0,4.0,3.0,40.0,60.0,40.0,60.0,31.0,29.0,38.8,Minnesota Wild


In [133]:
df = df.astype({'RW':int, 'ROW':int, 'SOW':int})
df.head()

,team,date,points,RW,ROW,SOW,goals,goals_against,power_play,penalty_kill,net_ppp,net_pkp,shots,shots_against,FOWp,opponent
0,Philadelphia Flyers,2010-10-07,2.0,1,1,0,3.0,2.0,25.0,80.0,25.0,100.0,27.0,31.0,39.3,Pittsburgh Penguins
1,Pittsburgh Penguins,2010-10-07,0.0,0,0,0,2.0,3.0,20.0,75.0,0.0,75.0,31.0,27.0,60.7,Philadelphia Flyers
2,Montréal Canadiens,2010-10-07,0.0,0,0,0,2.0,3.0,0.0,100.0,0.0,100.0,28.0,24.0,53.5,Toronto Maple Leafs
3,Toronto Maple Leafs,2010-10-07,2.0,1,1,0,3.0,2.0,0.0,100.0,0.0,100.0,24.0,28.0,46.5,Montréal Canadiens
4,Carolina Hurricanes,2010-10-07,2.0,1,1,0,4.0,3.0,40.0,60.0,40.0,60.0,31.0,29.0,38.8,Minnesota Wild
